# Deep Q Network

## 1. Setup

In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


env = gym.make('CartPole-v0').unwrapped

n_action = env.action_space.n
n_state = env.observation_space.shape[0]
print(n_state)
print(n_action)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion() # interactive mode on: 交互模式

## 2. Replay Memory

In [4]:
Experience = namedtuple('Experience', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = -1

    def push(self, *args):
        self.position = (self.position + 1) % self.capacity
        # if len(self.memory) < self.capacity:
        #     self.memory.append(None)
        self.memory[self.position] = Transition(*args) ## append a new experience

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self): ## len(experience)
        return len(self.memory)

## 3. Q-Network

In [5]:
class NN():
    def __init__(self, input_size, output_size):
        super(NN, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(input_size, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, output_size))

policy_net = NN(n_state,n_action).model
target_net = NN(n_state,n_action).model
target_net.load_state_dict(policy_net.state_dict()) ## copy policy to target

<All keys matched successfully>

## 4. Hyperparameters


In [ ]:
BATCH_SIZE = 128 #batch-train
GAMMA = 0.9 #reward-discount
EPS = 0.1 #epsilon-greedy
TARGET_UPDATE = 10 #policy to target